In [1]:
%load_ext autoreload

import os

import pandas as pd

print( os.getcwd() )
# change working directory
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util as du

/var/genie-in-the-box/src/ephemera/notebooks
/var/genie-in-the-box/src


In [2]:
# associate commands with files containing examples
browser_commands = {
    "search new tab": "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt",
    "search current tab": "/src/ephemera/prompts/data/synthetic-data-search-in-current-tab.txt",
    "search google new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-new-tab.txt",
    "search google current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-current-tab.txt",
    "search google scholar new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-new-tab.txt",
    "search google scholar current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-current-tab.txt",
}

In [3]:
for command in browser_commands.keys():
    print( os.path.exists( du.get_project_root() + browser_commands[ command ] ) )

True
True
True
True
True
True


In [106]:
raw_lines = du.get_file_as_list( du.get_project_root() + "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt", clean=True, randomize=True )[ 0:100 ]
raw_lines[ 0:10 ]

['New tab: search for SEARCH_TERMS',
 'Blow a search and dis play the rustling in another dab',
 'Joe a birch and display the rustles in a nu tub',
 'Grow a lurch and dismay the results in a gnu dab',
 'Search for SEARCH_TERMS, open in new tab',
 'Please show SEARCH_TERMS in a new tab',
 'Pull up a new tab and search SEARCH_TERMS',
 'Commence search for SEARCH_TERMS in another tab',
 'Scour for SEARCH_TERMS in another tab',
 'Stew a search, display the rustles in a nu dab']

In [107]:
def get_search_terms( requested_length ):
    
    # Load search terms file
    search_terms = du.get_file_as_list( du.get_project_root() + "/src/ephemera/prompts/data/search-terms.txt", lower_case=False, clean=True, randomize=True )
    
    # If we don't have enough search terms, append copies of the search term list until we do
    while requested_length > len( search_terms ):
        search_terms += search_terms
        
    # Truncate the search terms list to equal the requested len
    search_terms = search_terms[ :requested_length ]
    
    return search_terms

In [108]:
search_terms = get_search_terms( len( raw_lines ) )
search_terms[ 0:10]

['how to meditate',
 'AI for speech synthesis: What are the latest advancements in AI for speech synthesis?',
 'indie music',
 'best beaches in the world',
 'Cloud computing for AI',
 'Zero Division Error',
 'Normalization vs. Standardization',
 'latest breakthroughs in medicine',
 'Exploratory data analysis with Pandas',
 'Syntax Warning: Syntax issue warning']

In [109]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
command_choices 

"'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'"

In [157]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
question_template = """
Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 
{human_says}
The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
{response_format}
"""

human_says_template = """
<human>
    <voice-command>{voice_command}</voice-command>
</human>
"""

response_format = """
<response>
    <browser-command></browser-command>
    <args></args>
</response>

Remember: The first word of your response must be `<response>`
"""

answer_template = """
<response>
    <browser-command>{browser_command}</browser-command>
    <args>{args}</args>
</response>
"""

In [158]:
for line in question_template.split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 
{human_says}
The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
{response_format}


In [159]:
questions   = []
answers   = []
responses = []

for browser_command in browser_commands.keys():
    
    du.print_banner( browser_command, prepend_nl=True, end="\n" )
    
    raw_lines = du.get_file_as_list( du.get_project_root() + browser_commands[ browser_command ], clean=True )
    for line in raw_lines[ 0:100 ]: #[ 0:2 ]:#
        
        # get newly randomized search terms on every iteration
        for search in get_search_terms( len( raw_lines ) ):#[ 0:10 ]: #[ 0:2]:#:
            
            voice_command = line.replace( "SEARCH_TERMS", search )
            # print( voice_command )
            human_says    = human_says_template.format( voice_command=voice_command )
            question      = question_template.format( human_says=human_says, response_format=response_format, command_choices=command_choices )
            answer        = answer_template.format( browser_command=browser_command, args=search )
    
            questions.append( question )
            answers.append( answer )
    
            # du.print_banner( "Prompt" )
            # print( prompt )
            # du.print_banner( "Response" )
            # print( response )
            print( ".", end="" )
        
    print()


------------------------------------------------------------------------------------------------------------------------
- search new tab
------------------------------------------------------------------------------------------------------------------------
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [160]:
idx = 0

for line in questions[ 0 ].split( "\n" ):
    print( line )
    
for line in answers[ idx ].split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <voice-command>famous athletes new tab search</voice-command>
</human>

The standardized command that you translate MUST be returned

In [161]:
len( answers )

120000

In [162]:
len( questions )

120000

In [163]:
for line in questions[ 0 ].split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <voice-command>famous athletes new tab search</voice-command>
</human>

The standardized command that you translate MUST be returned

In [164]:
for line in answers[ 0 ].split( "\n" ):
    print( line )


<response>
    <browser-command>search new tab</browser-command>
    <args>famous athletes</args>
</response>


In [165]:
import random

def inject_space( line ):
    if random.random() < 0.5:
        index = random.randint( 0, len( line ) - 1 )
        return line[ :index ] + ' ' + line[ index: ]
    else:
        return line

In [166]:
qna_df = pd.DataFrame( { "question": questions, "answer": answers } )

# Perturb the answer to create a faked response  
qna_df[ "response" ] = qna_df[ "answer" ].apply( lambda cell: inject_space( cell ) )
qna_df.head()

,question,answer,response
0,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...
1,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...
2,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...
3,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...
4,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...


## Validate the structure of the xml response

In [124]:
!pip install xmlschema


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [167]:
from xmlschema import XMLSchema

xml_str_1 = """
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""
xml_str_2 = """
<response>
  <browser_command></browser_command>
  <args></args>
</response>
"""
xml_str_3 = """
Sure here's your command!
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xml_str_4 = """
```xml
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xsd_string = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="response">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="browser-command" type="xs:string"/>
        <xs:element name="args" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

def is_valid_xml( xml_str, schema ):
    
    try:
        schema.is_valid( xml_str )
        return True
    except Exception as e:
        # print( "Invalid XML: " + str( e ) )
        return False

schema   = XMLSchema( xsd_string )

print( is_valid_xml( xml_str_1, schema ) )  # Output: ???
print( is_valid_xml( xml_str_2, schema ) )  # Output: ???
print( is_valid_xml( xml_str_3, schema ) )  # Output: False
print( is_valid_xml( xml_str_4, schema ) )  # Output: False

True
True
False
False


In [168]:
qna_df[ "response_xml_is_valid" ] = qna_df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
qna_df.head()

,question,answer,response,response_xml_is_valid
0,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True
1,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True
2,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True
3,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True
4,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True


In [169]:
qna_df.response_xml_is_valid.mean()

0.7961333333333334

In [170]:
def contains_valid_xml_tag( xml_str, tag_name ):
    
    try:
        return "<" + tag_name + ">" in xml_str and "</" + tag_name + ">" in xml_str
    except Exception as e:
        return False
    
contains_valid_xml_tag( "```xml<response><browser-command></browser-command><args></args></response>", "browser-command")

True

## Validate the structure of the xml response

In [171]:
qna_df[ "contains_response" ]        = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "response" ) )
qna_df[ "contains_browser_command" ] = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "browser-command" ) )
qna_df[ "contains_args" ]            = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "args" ) )

In [172]:
print( qna_df[ "contains_response" ].mean() )
print( qna_df[ "contains_browser_command" ].mean() )
print( qna_df[ "contains_args" ].mean() )

0.9322166666666667
0.882275
0.9603083333333333


## Validate the response by removing all white space and comparing to the original answer

In [173]:
import re

xml_str = """
<response>
  <browser-command>Whos your favorite browser?</browser-command>
  <args>Bar browser</args>
</response>
"""

# Remove white space outside XML tags
xml_str = re.sub(r'>\s+<', '><', xml_str.strip())

print(xml_str)

<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>


In [174]:
def is_response_exact( response, answer ):
    
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

is_response_exact( "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [175]:
import lib.utils.util_xml as dux

def contains_correct_response_values( response, answer ):
    
    """Check to see if the most common formatting error (```xml) is hiding a correct response"""
    response = "<response>" + dux.get_value_by_xml_tag_name( response, "response", default_value="broken" ) + "</response>"
    if response == "broken":
        return False
    
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

contains_correct_response_values( "```xml<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [176]:
qna_df[ "response_is_correct" ] = qna_df.apply( lambda row: is_response_exact( row[ "response" ], row[ "answer" ] ), axis=1 )

In [177]:
qna_df.response_is_correct.mean()

0.5628666666666666

In [178]:
qna_df[ ( qna_df.response_xml_is_valid == True ) & ( qna_df.response_is_correct == False ) ].head( 10 )

,question,answer,response,response_xml_is_valid,contains_response,contains_browser_command,contains_args,response_is_correct
0,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
2,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
7,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
9,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
15,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
16,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new...,True,True,True,True,False
17,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
24,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
33,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
35,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,True,True,True,False


# What's the baseline behavior of the Vanilla model?

In [137]:
%autoreload
from huggingface_hub import InferenceClient
from lib.utils.util_stopwatch import Stopwatch
from lib.agents.agent import Agent

def query_llm_phind( prompt, model=Agent.PHIND_34B_v2, max_new_tokens=1024, temperature=0.25, top_k=10, top_p=0.9, debug=False, verbose=False, silent=True ):
    
    timer = Stopwatch( msg=f"Asking LLM [{model}]...".format( model ), silent=silent )
    
    # Get the TGI server URL for this context
    # default_url    = self.config_mgr.get( "tgi_server_url_phind", default=None )
    # tgi_server_url = du.get_tgi_server_url_for_this_context( default_url=default_url )
    
    client         = InferenceClient( "http://172.17.0.5:3000" )
    token_list     = [ ]
    ellipsis_count = 0
    
    if debug:
        for line in prompt.split( "\n" ):
            print( line )
    
    for token in client.text_generation(
        prompt, max_new_tokens=max_new_tokens, stream=True, temperature=temperature, top_k=top_k, top_p=top_p, stop_sequences=[ "</response>" ]
    ):
        if debug:
            print( token, end="" )
        else:
            if not silent: print( ".", end="" )
            ellipsis_count += 1
            if ellipsis_count == 120:
                ellipsis_count = 0
                print()
            
        token_list.append( token )
        
    response = "".join( token_list ).strip()
    
    timer.print( msg="Done!", use_millis=True, prepend_nl=True, end="\n" )
    tokens_per_second = len( token_list ) / ( timer.get_delta_ms() / 1000.0 )
    print( f"Tokens per second [{round( tokens_per_second, 1 )}]" )
            
    if debug:
        print( f"Token list length [{len( token_list )}]" )
        if verbose:
            for line in response.split( "\n" ):
                print( line )
    
    
    return response

In [35]:
import openai

def query_llm_openai( preamble, query, model=Agent.GPT_3_5, debug=False, verbose=False ):
    
    openai.api_key = du.get_api_key( "openai" )
    
    timer = Stopwatch( msg=f"Asking LLM [{model}]...".format( model ) )
    
    response = openai.chat.completions.create(
        model=model,
        messages=[
            { "role": "system", "content": preamble },
            { "role": "user", "content": query }
        ],
        temperature=0,
        max_tokens=2000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    timer.print( "Done!", use_millis=True )
    if debug and verbose:
        # print( json.dumps( response.to_dict(), indent=4 ) )
        print( response )
    
    return response.choices[ 0 ].message.content.strip()

In [142]:
!pip show openai

Name: openai
Version: 1.3.5
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, tqdm, typing-extensions
Required-by: 


In [182]:
call_counter = 0
# rows         = df.shape[ 0 ]

def get_response_to_question( question, rows ):
    
    global call_counter
    
    call_counter += 1
    print( f"On call [{call_counter:03d}] out of [{rows}] = [{round( call_counter / rows * 100.0, 1 )}%]... ", end="" )
    
    return query_llm_phind( question )

In [183]:
def validate( df ):
    
    rows = df.shape[ 0 ]
    global call_counter
    call_counter = 0
    
    df[ "response" ]                    = df[ "question" ].apply( lambda cell: get_response_to_question( cell, rows ) )
    
    # Validate the structure and content of the xml response
    df[ "response_xml_is_valid" ]       = df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
    df[ "contains_response" ]           = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "response" ) )
    df[ "contains_browser_command" ]    = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "browser-command" ) )
    df[ "contains_args" ]               = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "args" ) )
    df[ "response_is_exact" ]           = df.apply( lambda row: is_response_exact( row[ "response" ], row[ "answer" ] ), axis=1 )
    df[ "response_has_correct_values" ] = df.apply( lambda row: contains_correct_response_values( row[ "response" ], row[ "answer" ] ), axis=1 )
    
    return df


In [184]:
def print_stats( df ):

    du.print_banner( "Stats" )
    print( f"               Is valid xml {df.response_xml_is_valid.mean() * 100:.1f}%" )
    print( f"          Contains response {df.contains_response.mean() * 100:.1f}%" )
    print( f"   Contains browser command {df.contains_browser_command.mean() * 100:.1f}%" )
    print( f"              Contains args {df.contains_args.mean() * 100:.1f}%" )
    print( f"          Response is exact {df.response_is_exact.mean() * 100:.1f}%" )
    print( f"Response has correct values {df.response_has_correct_values.mean() * 100:.1f}%" )
    

In [186]:
# ! pip install scikit-learn

In [192]:
from sklearn.model_selection import train_test_split

sampled_df = qna_df[ [ "question", "answer" ] ].sample( 1000, random_state=42 ).copy()

# Split the dataframe into train and (test+validate)
train_df, test_validate_df = train_test_split( sampled_df, test_size=0.2, random_state=42 )

# Then split (test+validate) into test and validate
test_df, validate_df = train_test_split( test_validate_df, test_size=0.5, random_state=42 )


In [193]:
print( train_df.shape )
print( test_df.shape )
print( validate_df.shape )

(800, 2)
(100, 2)
(100, 2)


In [ ]:
validate_df = validate( validate_df )

print_stats( validate_df )

# First run before attempting to force valid XML
# ------------------------------------------------------------------------------------------------------------------------
# - Stats
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 0.0%
#           Contains response 98.0%
#    Contains browser command 98.0%
#               Contains args 98.0%
#           Response is exact 0.0%
# Response has correct values 43.0%

# after attempting to force the first token output: <response>
# ------------------------------------------------------------------------------------------------------------------------
# - Stats
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 0.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 0.0%
# Response has correct values 39.0%

In [191]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 61754 to 88409
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  800 non-null    object
 1   answer    800 non-null    object
dtypes: object(2)
memory usage: 18.8+ KB


In [198]:
path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-train.jsonl"
train_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-validate.jsonl"
validate_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-test.jsonl"
test_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )